In [ ]:
def make_grid_search(estimator, param_grid, cv=5):

    from sklearn.model_selection import GridSearchCV

    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        cv=cv,
        scoring="balanced_accuracy",
        n_jobs=-1,
    )

    return grid_search
  
  
  
  
  
def train_user_model(user_model, user_param_grid, cv=5):

    from sklearn.metrics import balanced_accuracy_score

    data, target = load_data()

    x_train, x_test, y_train, y_test = make_train_test_split(
        x=data,
        y=target,
    )

    pipeline = make_pipeline(
        estimator=user_model,
    )

    estimator = make_grid_search(
        estimator=pipeline,
        param_grid=user_param_grid,
        cv=cv,
    )

    estimator.fit(x_train, y_train)

    best_estimator = load_estimator()

    if best_estimator is not None:

        saved_balanced_accuracy = balanced_accuracy_score(
            y_true=y_test, y_pred=best_estimator.predict(x_test)
        )

        current_balanced_accuracy = balanced_accuracy_score(
            y_true=y_test, y_pred=estimator.predict(x_test)
        )

        if current_balanced_accuracy < saved_balanced_accuracy:
            estimator = best_estimator

    save_estimator(estimator)
    
    

def train_logistic_regression():

    from sklearn.linear_model import LogisticRegression

    train_user_model(
        user_model=LogisticRegression(max_iter=1000),
        user_param_grid={
            "transformer__norm": ["l1", "l2", None],
            "transformer__use_idf": [True, False],
            "transformer__smooth_idf": [True, False],
        },
        cv=5,
    )


train_logistic_regression()




def train_mlp_classifier():

    from sklearn.neural_network import MLPClassifier

    train_user_model(
        user_model=MLPClassifier(max_iter=10000),
        user_param_grid={
            "transformer__norm": ["l1", "l2", None],
            "transformer__use_idf": [True, False],
            "transformer__smooth_idf": [True, False],
            "estimator__hidden_layer_sizes": [(1,), (5,), (5, 5)],
            "estimator__solver": ["adam"],
            "estimator__learning_rate_init": [0.01, 0.001, 0.0001],
        },
        cv=5,
    )


train_mlp_classifier()